# hVAE

In [3]:
from src.hvae import hVAE

PATH = "path"

hvae = hVAE.load(PATH)
hvae.eval()
hvae.generate(10)

TypeError: load() missing 1 required positional argument: 'path'

In [ ]:
from torch import nn

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)


vae = VAE()
for p in vae.parameters():
    print(p.shape)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def visualize_graph():
    G = nx.DiGraph()

    encoder_edges = [('x', 'z1'), ('y', 'z1'), ('x', 'y')]
    decoder_edges = [('z1', 'x'), ('y', 'z1')]

    # use the same node list for both nx.draw calls
    nodes = ['x', 'y', 'z1']

    # specify the positions of the nodes
    pos = {'x': (0, 0), 'z1': (0, 1), 'y': (0, 2)}

    nx.draw(G, nodelist=nodes, pos=pos, edgelist=encoder_edges, edge_color="r", width=2, node_size=2000, with_labels=True, node_color="lightblue", ax=plt.gca(), arrowstyle="->", connectionstyle="arc3,rad=0.2")

    # draw the blue edges with labels and arrows on the same axis with negative curvature
    nx.draw(G, nodelist=nodes, pos=pos, edgelist=decoder_edges, edge_color="b", width=2, node_size=2000, with_labels=True, node_color="lightblue", ax=plt.gca(), arrowstyle="->")
    # show the plot
    plt.show()


In [ ]:
visualize_graph()

In [1]:
from hparams import get_hparams

hparams = get_hparams()
model = hparams.model_params.model()

In [1]:
from test import main
main()

Loading from experiments/TDVAE/2023-08-10_22-13-14/checkpoints/checkpoint-25.pth


2023-08-10 22:30:39,857 - INFO - Model Checkpoint is loaded
2023-08-10 22:30:42,297 - INFO - Validation Stats| Reconstruction Loss 37.4010 | KL Div 202005.3125 |NELBO 202042.718750 | SSIM: 0.144424


In [1]:
from train import main
main()

2023-08-10 22:13:14,116 - INFO - Train step generator trainable params 18.361m.
2023-08-10 22:13:14,168 - INFO - (0, ('Time/Step (sec)', 0.05), ('Reconstruction Loss', 18.738), ('KL loss', 1425.725), ('ELBO', 18.837), ('average KL loss', 2919.886), ('Beta', 0.0001), ('N° active groups', 2), ('GradNorm', 45676.4), ('GradSkipCount', 1.0))
2023-08-10 22:13:14,169 - INFO - 
-----------------------------------------------------------------------------------------
2023-08-10 22:13:14,175 - INFO - Train Stats for global_step 0 | NELBO 18.836956024169922 | SSIM: 0.14734524488449097
2023-08-10 22:13:16,515 - INFO - Validation Stats for global_step 0 | Reconstruction Loss 37.4009 | KL Div 950246.0625 |NELBO 950283.437500 | SSIM: 0.144617
2023-08-10 22:13:16,516 - INFO - Logging to Tensorboard..
2023-08-10 22:13:19,352 - INFO - Saved checkpoint for global_step 0 to experiments/TDVAE/2023-08-10_22-13-14/checkpoints/checkpoint-0.pth
2023-08-10 22:13:19,354 - INFO - 
--------------------------------

KeyboardInterrupt: 

In [1]:
from synthesize import main

main()

Loading from experiments/TDVAE/2023-08-10_22-13-14/checkpoints/checkpoint-25.pth


2023-08-10 22:36:46,249 - INFO - Model Checkpoint is loaded


AssertionError: 